In [11]:
class Entry:

    def __init__(self):
        self.valid = 0
        self.tag = 0
        self.ref = 0

    def display(self):
        if not self.valid:
            print("| X ", end="")
        else:
            print(f"| {str(self.tag)} ", end="")

class Cache:

    def __init__(self, _num_entries, _assoc):
        self.num_entries = _num_entries
        self.assoc = _assoc
        self.num_sets = (int) (_num_entries / _assoc)
        self.list_of_list_of_entries = [[Entry() for x in range(self.num_sets)] for x in range(self.assoc)]

        for set_, row in enumerate(self.list_of_list_of_entries):
            for entry in row:
                entry.tag = 0
                entry.valid = 0
                entry.ref = set_

        print("Cache created:")
        print("\t" + str(self.assoc) + "-way associative with " + str(self.num_entries) + " entries")
        print("\t" + str(self.num_sets) + " sets per way")

        self.display()

    def get_index(self, addr) -> int:
        return addr % self.num_sets

    def get_tag(self, addr) -> int:
        return (int) (addr / self.num_sets)

    def hit(self, addr) -> bool:

        # Get the set that we are referencing
        index = self.get_index(addr)

        # Iterate our sets
        for i,list_of_entries in enumerate(self.list_of_list_of_entries):

            # Get the entry from that set given the index
            entry = list_of_entries[index]

            # Is the entry valid?
            if entry.valid:

                # Does the tag match?
                if (entry.tag == self.get_tag(addr)):
                    ## this is a cache hit; adjust LRU ref counter

                    ## assoc - 1 = most recent
                    for sub_list_of_entries in self.list_of_list_of_entries:

                        sub_entry = sub_list_of_entries[index]

                        # If entry is more recent than current ref then make it older
                        if (sub_entry.ref > entry.ref):
                            sub_entry.ref = sub_entry.ref - 1

                    # Make target entry the newest entry
                    entry.ref = self.assoc - 1

                    print("--- [Cache Hit] Found : " + str(addr) + " in Set " + str(i) + ", Index " + str(index))

                    return True

                else:

                    # Tag did not match
                    print("--- [Cache Miss] Did not find data for addr: " + str(addr))
                    print("        Tag mismatch: Addr Tag = " + str(self.get_tag(addr)) + " Stored Tag: " + str(entry.tag))
                    return False

            else:

                # Entry was not valid
                print("--- [Cache Miss] Did not find data for addr: " + str(addr))
                print("        Cache block has invalid entry ")

        return False


    def update(self, addr):

        # Display whether it is a hit or a miss
        self.hit(addr)

        # Get which set we are referencing
        index = self.get_index(addr)

        # Iterate through all the sets
        for list_of_entries in self.list_of_list_of_entries:

            # Get the entry from the set
            entry = list_of_entries[index]

            # Skip entries with valid refs
            if not entry.ref:

                # (1) Set the valid bit
                entry.valid = 1

                # (2) Set the ref bit
                # If entry is more recent than current ref then make it older
                for sub_list_of_entries in self.list_of_list_of_entries:
                    if (sub_list_of_entries[index].ref > entry.ref):
                        sub_list_of_entries[index].ref = sub_list_of_entries[index].ref - 1

                # Make target entry the newest entry
                entry.ref = self.assoc - 1

                # (3) Set the tag
                entry.tag = self.get_tag(addr)

                # Break after we found an entry to write from all the sets
                break


    def display(self):

        print("************ Cache ************")

        # Display top line
        print("+", end="")
        for j in range(self.assoc):
            print("---+", end="")
        print()

        # Display each set
        for i in range(self.num_sets):

            # Print the entries in the set
            for j in range(self.assoc):
                self.list_of_list_of_entries[j][i].display()
            print("|")

            # Print the bottom line for that set
            print("+", end="")
            for j in range(self.assoc):
                print("---+", end="")
            print()

        print("********************************")


In [12]:
cache = Cache(8, 1)

Cache created:
	1-way associative with 8 entries
	8 sets per way
************ Cache ************
+---+
| X |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
********************************


In [13]:
cache.get_index(3)

3

In [14]:
cache.get_index(17)

1

In [15]:
cache.get_index(24)

0

In [16]:
cache.get_tag(3)

0

In [17]:
cache.get_tag(17)

2

In [18]:
cache.get_tag(24)

3

In [19]:
cache.update(4)
cache.update(3)
cache.update(17)
cache.update(24)

--- [Cache Miss] Did not find data for addr: 4
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 3
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 17
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 24
        Cache block has invalid entry 


In [20]:
cache.display()

************ Cache ************
+---+
| 3 |
+---+
| 2 |
+---+
| X |
+---+
| 0 |
+---+
| 0 |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
********************************


In [21]:
cache.update(3)
cache.update(4)

--- [Cache Hit] Found : 3 in Set 0, Index 3
--- [Cache Hit] Found : 4 in Set 0, Index 4


In [22]:
cache.display()

************ Cache ************
+---+
| 3 |
+---+
| 2 |
+---+
| X |
+---+
| 0 |
+---+
| 0 |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
********************************


In [23]:
cache.update(11)

--- [Cache Miss] Did not find data for addr: 11
        Tag mismatch: Addr Tag = 1 Stored Tag: 0


In [24]:
cache.get_index(3)

3

In [25]:
cache.get_index(11)

3

In [26]:
cache.display()

************ Cache ************
+---+
| 3 |
+---+
| 2 |
+---+
| X |
+---+
| 1 |
+---+
| 0 |
+---+
| X |
+---+
| X |
+---+
| X |
+---+
********************************


In [27]:
cache.get_tag(11)

1

In [28]:
cache.update(3)

--- [Cache Miss] Did not find data for addr: 3
        Tag mismatch: Addr Tag = 0 Stored Tag: 1


In [29]:
cache = Cache(8,2)

Cache created:
	2-way associative with 8 entries
	4 sets per way
************ Cache ************
+---+---+
| X | X |
+---+---+
| X | X |
+---+---+
| X | X |
+---+---+
| X | X |
+---+---+
********************************


In [30]:
cache.update(3)
cache.update(17)
cache.update(24)

--- [Cache Miss] Did not find data for addr: 3
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 3
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 17
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 17
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 24
        Cache block has invalid entry 
--- [Cache Miss] Did not find data for addr: 24
        Cache block has invalid entry 


In [31]:
cache.display()

************ Cache ************
+---+---+
| 6 | X |
+---+---+
| 4 | X |
+---+---+
| X | X |
+---+---+
| 0 | X |
+---+---+
********************************


In [32]:
cache.update(3)
cache.update(11)
cache.update(3)

--- [Cache Hit] Found : 3 in Set 0, Index 3
--- [Cache Miss] Did not find data for addr: 11
        Tag mismatch: Addr Tag = 2 Stored Tag: 0
--- [Cache Miss] Did not find data for addr: 3
        Tag mismatch: Addr Tag = 0 Stored Tag: 2


In [33]:
cache.display()

************ Cache ************
+---+---+
| 6 | X |
+---+---+
| 4 | X |
+---+---+
| X | X |
+---+---+
| 2 | 0 |
+---+---+
********************************
